In [1]:
import sys

sys.path.append("../")

import numpy as np
import pandas as pd

# from src.dataloader_ import *
# from src.network_ import *
from src.utils import *

import os
import sys
import gc
import pickle
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer, KNNImputer
from pytorch_tabnet.tab_model import TabNetRegressor

train_series_dir = "../../../inputs/series_train.parquet/"
test_series_dir = "../../../inputs/series_test.parquet/"

data_dic_path = "../../../inputs/data_dictionary.csv"
sample_submission_path = "../../../inputs/sample_submission.csv"
train_path = "../../../inputs/train.csv"
test_path = "../../../inputs/test.csv"

train = pd.read_csv(train_path)
test = pd.read_csv(test_path)
sample_submission = pd.read_csv(sample_submission_path)
data_dic = pd.read_csv(data_dic_path)

import os
import random

import numpy as np
import torch


def seed_torch(seed=1029):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True


nb_name = os.path.basename(os.getcwd())  # notebook name
seed_torch(seed=42)

In [2]:
# データの前処理

from sklearn.impute import SimpleImputer, KNNImputer

add_features = [
    "BMI_Age",
    "Internet_Hours_Age",
    "BMI_Internet_Hours",
    "BFP_BMI",
    "FFMI_BFP",
    "FMI_BFP",
    "LST_TBW",
    "BFP_BMR",
    "BFP_DEE",
    "BMR_Weight",
    "DEE_Weight",
    "SMM_Height",
    "Muscle_to_Fat",
    "Hydration_Status",
    "ICW_TBW",
]

double_columns = [
    "FGC-FGC_SRR_Zone",
    "BIA-BIA_SMM",
    "Physical-Waist_Circumference",
    "BIA-BIA_FFMI",
    "FGC-FGC_CU",
    "PreInt_EduHx-computerinternet_hoursday",
    "BIA-BIA_ECW",
    "FGC-FGC_CU_Zone",
    "FGC-FGC_SRL_Zone",
    "BIA-BIA_DEE",
    "Physical-Weight",
    "Fitness_Endurance-Time_Mins",
    "FGC-FGC_SRR",
    "SDS-SDS_Total_T",
    "FGC-FGC_PU",
    "BIA-BIA_FFM",
    "FGC-FGC_TL_Zone",
    "Physical-BMI",
    "Physical-Systolic_BP",
    "Physical-HeartRate",
    "BIA-BIA_ICW",
    "Physical-Height",
    "FGC-FGC_SRL",
    "BIA-BIA_BMC",
    "Fitness_Endurance-Time_Sec",
    "BIA-BIA_Frame_num",
    "Basic_Demos-Age",
    "FGC-FGC_GSND_Zone",
    "Basic_Demos-Sex",
    "FGC-FGC_GSND",
    "BIA-BIA_LST",
    "FGC-FGC_TL",
    "BIA-BIA_BMI",
    "BIA-BIA_FMI",
    "PAQ_C-PAQ_C_Total",
    "BIA-BIA_Activity_Level_num",
    "FGC-FGC_GSD",
    "BIA-BIA_BMR",
    "BIA-BIA_Fat",
    "SDS-SDS_Total_Raw",
    "CGAS-CGAS_Score",
    "FGC-FGC_PU_Zone",
    "BIA-BIA_LDM",
    "Fitness_Endurance-Max_Stage",
    "PAQ_A-PAQ_A_Total",
    "BIA-BIA_TBW",
    "FGC-FGC_GSD_Zone",
    "Physical-Diastolic_BP",
]


def feature_engineering(df):
    # season_cols = [col for col in df.columns if "Season" in col]
    # df = df.drop(season_cols, axis=1)
    df["BMI_Age"] = df["Physical-BMI"] * df["Basic_Demos-Age"]
    df["Internet_Hours_Age"] = (
        df["PreInt_EduHx-computerinternet_hoursday"] * df["Basic_Demos-Age"]
    )
    df["BMI_Internet_Hours"] = (
        df["Physical-BMI"] * df["PreInt_EduHx-computerinternet_hoursday"]
    )
    df["BFP_BMI"] = df["BIA-BIA_Fat"] / df["BIA-BIA_BMI"]
    df["FFMI_BFP"] = df["BIA-BIA_FFMI"] / df["BIA-BIA_Fat"]
    df["FMI_BFP"] = df["BIA-BIA_FMI"] / df["BIA-BIA_Fat"]
    df["LST_TBW"] = df["BIA-BIA_LST"] / df["BIA-BIA_TBW"]
    df["BFP_BMR"] = df["BIA-BIA_Fat"] * df["BIA-BIA_BMR"]
    df["BFP_DEE"] = df["BIA-BIA_Fat"] * df["BIA-BIA_DEE"]
    df["BMR_Weight"] = df["BIA-BIA_BMR"] / df["Physical-Weight"]
    df["DEE_Weight"] = df["BIA-BIA_DEE"] / df["Physical-Weight"]
    df["SMM_Height"] = df["BIA-BIA_SMM"] / df["Physical-Height"]
    df["Muscle_to_Fat"] = df["BIA-BIA_SMM"] / df["BIA-BIA_FMI"]
    df["Hydration_Status"] = df["BIA-BIA_TBW"] / df["Physical-Weight"]
    df["ICW_TBW"] = df["BIA-BIA_ICW"] / df["BIA-BIA_TBW"]

    return df


train = feature_engineering(train)
train = train.replace([np.inf, -np.inf], np.nan)
for add_ in add_features:
    train[add_] = train[add_].fillna(0.0)
train = train.dropna(thresh=10, axis=0)

test = feature_engineering(test)
test = test.replace([np.inf, -np.inf], np.nan)
for add_ in add_features:
    test[add_] = test[add_].fillna(0.0)
test = test.dropna(thresh=10, axis=0)

***Tabnet***

In [3]:
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from pytorch_tabnet.callbacks import Callback
import os
import torch
from pytorch_tabnet.callbacks import Callback


class TabNetWrapper(BaseEstimator, RegressorMixin):
    def __init__(self, **kwargs):
        self.model = TabNetRegressor(**kwargs)
        self.kwargs = kwargs
        self.imputer = SimpleImputer(strategy="median")
        self.best_model_path = "best_tabnet_model.pt"

    def fit(self, X, y):
        # Handle missing values
        X_imputed = self.imputer.fit_transform(X)

        if hasattr(y, "values"):
            y = y.values

        # Create internal validation set
        X_train, X_valid, y_train, y_valid = train_test_split(
            X_imputed, y, test_size=0.2, random_state=42
        )

        # Train TabNet model
        history = self.model.fit(
            X_train=X_train,
            y_train=y_train.reshape(-1, 1),
            eval_set=[(X_valid, y_valid.reshape(-1, 1))],
            eval_name=["valid"],
            eval_metric=["mse"],
            max_epochs=200,
            patience=20,
            batch_size=1024,
            virtual_batch_size=128,
            num_workers=0,
            drop_last=False,
            callbacks=[
                TabNetPretrainedModelCheckpoint(
                    filepath=self.best_model_path,
                    monitor="valid_mse",
                    mode="min",
                    save_best_only=True,
                    verbose=True,
                )
            ],
        )

        # Load the best model
        if os.path.exists(self.best_model_path):
            self.model.load_model(self.best_model_path)
            os.remove(self.best_model_path)  # Remove temporary file

        return self

    def predict(self, X):
        X_imputed = self.imputer.transform(X)
        return self.model.predict(X_imputed).flatten()

    def __deepcopy__(self, memo):
        # Add deepcopy support for scikit-learn
        cls = self.__class__
        result = cls.__new__(cls)
        memo[id(self)] = result
        for k, v in self.__dict__.items():
            setattr(result, k, deepcopy(v, memo))
        return result


# TabNet hyperparameters
TabNet_Params = {
    "n_d": 64,  # Width of the decision prediction layer
    "n_a": 64,  # Width of the attention embedding for each step
    "n_steps": 5,  # Number of steps in the architecture
    "gamma": 1.5,  # Coefficient for feature selection regularization
    "n_independent": 2,  # Number of independent GLU layer in each GLU block
    "n_shared": 2,  # Number of shared GLU layer in each GLU block
    "lambda_sparse": 1e-4,  # Sparsity regularization
    "optimizer_fn": torch.optim.Adam,
    "optimizer_params": dict(lr=2e-2, weight_decay=1e-5),
    "mask_type": "entmax",
    "scheduler_params": dict(mode="min", patience=10, min_lr=1e-5, factor=0.5),
    "scheduler_fn": torch.optim.lr_scheduler.ReduceLROnPlateau,
    "verbose": 1,
    "device_name": "cuda" if torch.cuda.is_available() else "cpu",
}


class TabNetPretrainedModelCheckpoint(Callback):
    def __init__(
        self, filepath, monitor="val_loss", mode="min", save_best_only=True, verbose=1
    ):
        super().__init__()  # Initialize parent class
        self.filepath = filepath
        self.monitor = monitor
        self.mode = mode
        self.save_best_only = save_best_only
        self.verbose = verbose
        self.best = float("inf") if mode == "min" else -float("inf")

    def on_train_begin(self, logs=None):
        self.model = self.trainer  # Use trainer itself as model

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        current = logs.get(self.monitor)
        if current is None:
            return

        # Check if current metric is better than best
        if (self.mode == "min" and current < self.best) or (
            self.mode == "max" and current > self.best
        ):
            if self.verbose:
                print(
                    f"\nEpoch {epoch}: {self.monitor} improved from {self.best:.4f} to {current:.4f}"
                )
            self.best = current
            if self.save_best_only:
                self.model.save_model(self.filepath)  # Save the entire model

In [4]:
import warnings

warnings.filterwarnings("ignore")
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import (
    VotingRegressor,
    RandomForestRegressor,
    GradientBoostingRegressor,
)


def extract_features(df):
    return df[double_columns + add_features]


seed = 42
oof = []
cv_scores = []
y = None

# Model parameters for LightGBM
Params = {
    "learning_rate": 0.046,
    "max_depth": 12,
    "num_leaves": 478,
    "min_data_in_leaf": 13,
    "feature_fraction": 0.893,
    "bagging_fraction": 0.784,
    "bagging_freq": 4,
    "lambda_l1": 10,  # Increased from 6.59
    "lambda_l2": 0.01,  # Increased from 2.68e-06
}
XGB_Params = {
    "learning_rate": 0.05,
    "max_depth": 6,
    "n_estimators": 200,
    "subsample": 0.8,
    "colsample_bytree": 0.8,
    "reg_alpha": 1,  # Increased from 0.1
    "reg_lambda": 5,  # Increased from 1
    "random_state": seed,
}

CatBoost_Params = {
    "learning_rate": 0.05,
    "depth": 6,
    "iterations": 200,
    "random_seed": seed,
    "verbose": 0,
    "l2_leaf_reg": 10,  # Increase this value
}
from sklearn.pipeline import Pipeline

for fold in range(5):
    Light = LGBMRegressor(**Params, random_state=seed, verbose=-1, n_estimators=300)
    XGB_Model = XGBRegressor(**XGB_Params)
    CatBoost_Model = CatBoostRegressor(**CatBoost_Params)
    TabNet_Model = TabNetWrapper(**TabNet_Params)  # New

    SEED = 42
    imputer = SimpleImputer(strategy="median")
    model = VotingRegressor(
        estimators=[
            (
                "lgb",
                Pipeline(
                    steps=[
                        ("imputer", imputer),
                        ("regressor", LGBMRegressor(random_state=SEED)),
                    ]
                ),
            ),
            (
                "xgb",
                Pipeline(
                    steps=[
                        ("imputer", imputer),
                        ("regressor", XGBRegressor(random_state=SEED)),
                    ]
                ),
            ),
            (
                "cat",
                Pipeline(
                    steps=[
                        ("imputer", imputer),
                        (
                            "regressor",
                            CatBoostRegressor(random_state=SEED, silent=True),
                        ),
                    ]
                ),
            ),
            (
                "rf",
                Pipeline(
                    steps=[
                        ("imputer", imputer),
                        ("regressor", RandomForestRegressor(random_state=SEED)),
                    ]
                ),
            ),
            (
                "gb",
                Pipeline(
                    steps=[
                        ("imputer", imputer),
                        ("regressor", GradientBoostingRegressor(random_state=SEED)),
                    ]
                ),
            ),
        ]
    )

    # model = LGBMRegressor(**Params, random_state=seed, verbose=-1, n_estimators=300)

    with open(f"../divided-datasets/fold_train_ids_{fold}.pkl", "rb") as f:
        fold_train_ids = pickle.load(f)

    with open(f"../divided-datasets/fold_valid_ids_{fold}.pkl", "rb") as f:
        fold_valid_ids = pickle.load(f)

    train_fold = train[train["id"].isin(fold_train_ids)].reset_index(drop=True)
    valid_fold = train[train["id"].isin(fold_valid_ids)].reset_index(drop=True)

    mode = "drop"

    if mode == "impute":
        numeric_cols = train.select_dtypes(include=["float64", "int64"]).columns
        imputer = KNNImputer(n_neighbors=5)
        train_fold[numeric_cols] = imputer.fit_transform(train_fold[numeric_cols])
        test[numeric_cols] = imputer.transform(test[numeric_cols])
    elif mode == "drop":
        train_fold = train_fold[train_fold["sii"].notnull()].reset_index(drop=True)

    train_fold_x = extract_features(train_fold)
    valid_fold_x = extract_features(valid_fold)

    train_fold_y = train_fold["sii"].astype(int)
    valid_fold_y = valid_fold["sii"].astype(int)

    print(train_fold_x.shape, valid_fold_x.shape)

    model.fit(train_fold_x, train_fold_y)

    # save model
    with open(f"./assets/model05_{fold}.pkl", "wb") as f:
        pickle.dump(model, f)

    train_pred = model.predict(train_fold_x)
    valid_pred = model.predict(valid_fold_x)

    train_kappa = quadratic_weighted_kappa(
        train_fold_y, train_pred.round(0).astype(int)
    )
    val_kappa = quadratic_weighted_kappa(valid_fold_y, valid_pred.round(0).astype(int))
    cv_scores.append(val_kappa)

    print(
        f"Fold {fold+1} - Train QWK: {train_kappa:.4f}, Validation QWK: {val_kappa:.4f}"
    )

    for i, id_ in enumerate(fold_valid_ids):
        oof.append({"id": id_, "sii": valid_pred[i]})

    if y is None:
        y = valid_fold[["id", "sii"]]
    else:
        y = pd.concat([y, valid_fold[["id", "sii"]]], axis=0).reset_index(drop=True)

oof = pd.DataFrame(oof)

KappaOPtimizer = minimize(
    evaluate_predictions,
    x0=[0.5, 1.5, 2.5],
    args=(y["sii"].astype(int), oof["sii"]),
    method="Nelder-Mead",
)

oof_tuned = threshold_Rounder(oof["sii"], KappaOPtimizer.x)
tKappa = quadratic_weighted_kappa(y["sii"], oof_tuned)
print(f"CV: {np.mean(cv_scores):.4f}")
print(f"tuned Kappa: {tKappa:.3f}")  # tuned Kappa: 0.451

(2188, 63) (548, 63)
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002532 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 9439
[LightGBM] [Info] Number of data points in the train set: 2188, number of used features: 63
[LightGBM] [Info] Start training from score 0.579068
Fold 1 - Train QWK: 0.9164, Validation QWK: 0.3351
(2189, 63) (547, 63)
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000664 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 9428
[LightGBM] [Info] Number of data points in the train set: 2189, number of used features: 63
[LightGBM] [Info] Start training from score 0.581087
Fold 2 - Train QWK: 0.9156, Validation QWK: 0.3863
(2189, 63) (547, 63)
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.0

In [5]:
oof.to_csv("./oof/oof.csv", index=False)